# Audio classification through image classification:

This notebook shows how to use a pre-trained resent with additional Fully Convolutional Net to predict which call is good or bad. 

Advantage:

- The advantage of this method is that it does not require any audio file feature engineering and nor it relies on the text that is associated with each audio recording. This means it can be faster and much cheaper.

Disadvantage: 

- The accuracy on the current dataset is around 60% which is lower than both the MFCC and NLP based approaches. But as will be shown in the following this model, and many other models that I tested, start overfitting as the training process continues which indicates that more data will improves the accuracy of this model.   
 


In [1]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; 

import importlib
importlib.reload(utils)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def save_array(fname, arr): 
    c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

from utils import *
current_dir = os.getcwd()
print ('current folder: ',current_dir)
model_path = current_dir+'/Data/crop_data/models'
print ('model path: ',model_path)

current folder:  /home/sohrab/MaestroQA_ffast_ai
model path:  /home/sohrab/MaestroQA_ffast_ai/Data/crop_data/models


Using cuDNN version 6021 on context None
Mapped name None to device cuda: Quadro P6000 (0000:03:00.0)
Using Theano backend.


In [2]:
import resnet50
from resnet50 import Resnet50

## Reading the victories images :

In [3]:
trn_data_1 = load_array(current_dir+'/Data/crop_data/models/train_data_big.bc')
val_data_1 = load_array(current_dir+'/Data/crop_data/models/valid_data_big.bc')

trn_labels_1 = load_array(current_dir+'/Data/crop_data/models/trn_labels_big.bc')
val_labels_1 = load_array(current_dir+'/Data/crop_data/models/val_labels_big.bc')

print (trn_data_1.shape,val_data_1.shape, trn_labels_1.shape, val_labels_1.shape )

(10326, 3, 430, 1246) (1999, 3, 430, 1246) (10326, 2) (1999, 2)


### Defining the Resnet50 model:

In [4]:
rn0 = Resnet50(include_top=False , size= (430,1246)).model

/home/sohrab/anaconda3/lib/python3.5/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 430, 1246)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/home/sohrab/MaestroQA_ffast_ai/resnet50.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), name="conv1", strides=(2, 2))`
  x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1')(x)


In [5]:
# Extracting feature from the pre-trained model
trn_features = rn0.predict(trn_data_1, batch_size=128, verbose=1)
val_features = rn0.predict(val_data_1, batch_size=128, verbose=1)

1999/1999 [==============================] - 77s 38ms/step


### Adding the Fully Convolutional Net:

In [6]:
nf=128; p=0
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=(2048, 14, 39)),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
#         MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
#         MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
#         MaxPooling2D((1,1)),
        Convolution2D(2,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [7]:
lrg_model = Sequential(get_lrg_layers())

/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3), padding="same")`


In [8]:
lrg_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
batch_size=32
lrg_model.fit(trn_features, trn_labels_1, epochs=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/3
10326/10326 [==============================] - 36s 4ms/step - loss: 0.6990 - acc: 0.5530 - val_loss: 0.7061 - val_acc: 0.5508
Epoch 2/3
10326/10326 [==============================] - 37s 4ms/step - loss: 0.6767 - acc: 0.5809 - val_loss: 0.6908 - val_acc: 0.5448
Epoch 3/3
10326/10326 [==============================] - 37s 4ms/step - loss: 0.6627 - acc: 0.6011 - val_loss: 0.6843 - val_acc: 0.5548


In [10]:
batch_size=8
lrg_model.fit(trn_features, trn_labels_1, epochs=1, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/3
10326/10326 [==============================] - 39s 4ms/step - loss: 0.6626 - acc: 0.5976 - val_loss: 0.6737 - val_acc: 0.5808
Epoch 2/3
10326/10326 [==============================] - 39s 4ms/step - loss: 0.6465 - acc: 0.6195 - val_loss: 0.6940 - val_acc: 0.5683
Epoch 3/3
10326/10326 [==============================] - 40s 4ms/step - loss: 0.6297 - acc: 0.6365 - val_loss: 0.6980 - val_acc: 0.5753


In [11]:
batch_size=16
lrg_model.fit(trn_features, trn_labels_1, epochs=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/5
10326/10326 [==============================] - 40s 4ms/step - loss: 0.6088 - acc: 0.6611 - val_loss: 0.6861 - val_acc: 0.5808
Epoch 2/5
10326/10326 [==============================] - 40s 4ms/step - loss: 0.5830 - acc: 0.6840 - val_loss: 0.7114 - val_acc: 0.5888
Epoch 3/5
10326/10326 [==============================] - 40s 4ms/step - loss: 0.5442 - acc: 0.7190 - val_loss: 0.7815 - val_acc: 0.5493
Epoch 4/5
10326/10326 [==============================] - 40s 4ms/step - loss: 0.4975 - acc: 0.7514 - val_loss: 0.7881 - val_acc: 0.5763
Epoch 5/5
10326/10326 [==============================] - 40s 4ms/step - loss: 0.4428 - acc: 0.7847 - val_loss: 0.8099 - val_acc: 0.5553


In [12]:
batch_size=8
lrg_model.fit(trn_features, trn_labels_1, epochs=5, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/5
10326/10326 [==============================] - 44s 4ms/step - loss: 0.4740 - acc: 0.7689 - val_loss: 0.8702 - val_acc: 0.5538
Epoch 2/5
10326/10326 [==============================] - 44s 4ms/step - loss: 0.4101 - acc: 0.8067 - val_loss: 0.9432 - val_acc: 0.5693
Epoch 3/5
10326/10326 [==============================] - 43s 4ms/step - loss: 0.3606 - acc: 0.8374 - val_loss: 0.9643 - val_acc: 0.5703
Epoch 4/5
10326/10326 [==============================] - 44s 4ms/step - loss: 0.3131 - acc: 0.8611 - val_loss: 1.1057 - val_acc: 0.5503
Epoch 5/5
10326/10326 [==============================] - 43s 4ms/step - loss: 0.2783 - acc: 0.8790 - val_loss: 1.0925 - val_acc: 0.5728


In [14]:
nf=128; p=0.2
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=(2048, 14, 39)),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
         MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
         MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
         MaxPooling2D((1,1)),
        Convolution2D(2,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [15]:
lrg_model = Sequential(get_lrg_layers())

/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu")`
/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3), padding="same")`


In [16]:
lrg_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
batch_size=32
lrg_model.fit(trn_features, trn_labels_1, epochs=3, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/3
10326/10326 [==============================] - 34s 3ms/step - loss: 0.7090 - acc: 0.5335 - val_loss: 0.7061 - val_acc: 0.5073
Epoch 2/3
10326/10326 [==============================] - 34s 3ms/step - loss: 0.6775 - acc: 0.5846 - val_loss: 0.6971 - val_acc: 0.5448
Epoch 3/3
10326/10326 [==============================] - 35s 3ms/step - loss: 0.6553 - acc: 0.6096 - val_loss: 0.6933 - val_acc: 0.5668


In [18]:
batch_size=8
lrg_model.fit(trn_features, trn_labels_1, epochs=20, batch_size=batch_size, 
       validation_data=(val_features, val_labels_1))

Train on 10326 samples, validate on 1999 samples
Epoch 1/20
10326/10326 [==============================] - 41s 4ms/step - loss: 0.6765 - acc: 0.5792 - val_loss: 0.6888 - val_acc: 0.5388
Epoch 2/20
10326/10326 [==============================] - 39s 4ms/step - loss: 0.6503 - acc: 0.6155 - val_loss: 0.7132 - val_acc: 0.5563
Epoch 3/20
10326/10326 [==============================] - 40s 4ms/step - loss: 0.6239 - acc: 0.6519 - val_loss: 0.7051 - val_acc: 0.5638
Epoch 4/20
10326/10326 [==============================] - 39s 4ms/step - loss: 0.5710 - acc: 0.6905 - val_loss: 0.7303 - val_acc: 0.5643
Epoch 5/20
10326/10326 [==============================] - 40s 4ms/step - loss: 0.4988 - acc: 0.7508 - val_loss: 1.1591 - val_acc: 0.5198
Epoch 6/20
10326/10326 [==============================] - 39s 4ms/step - loss: 0.3971 - acc: 0.8190 - val_loss: 1.0384 - val_acc: 0.5518
Epoch 7/20
10326/10326 [==============================] - 40s 4ms/step - loss: 0.3037 - acc: 0.8657 - val_loss: 0.9946 - val_acc: